## Getting Started with the Maxar Geospatial Platform Tasking SDK

This sample workflow will guide a user through using the Tasking class of the Maxar Geospatial Platform SDK with real world examples. The process will go through:
* Viewing pipelines
* Creating a tasking request
* Viewing a tasking request
* Canceling a tasking request

#### Initialize the Maxar Geospatial Platform SDK Module

In [ ]:
from MGP_SDK.interface import Interface

try:
  interface = Interface() # if .MGP-config was created
except:
  interface = Interface('<user_name>','<user_password>', '<user_client_id>') # if .MGP-config was not created

#### Initialize the Tasking portion of the Maxar Geospatial Platform SDK

In [ ]:
mgp_tasking = interface.tasking_service

#### Getting a Pipeline

Firstly, a pipeline will need to be found to work the necessary information needed for the desired pipeline. If a user is unfamiliar with a pipeline and what is needed for the order to work, the user can use the `get_pipeline_details` function to determine what information needs to be passed in.

In [ ]:
mgp_ordering = interface.order_service

This workflow will use the `map-ready` pipeline with the namespace of `imagery`. With the pipeline identified, the response of the `get_pipeline_details` function can be narrowed down to show the necessary information needed to place a tasking request.

In [ ]:
map_pipeline = mgp_ordering.get_pipeline_details(namespace='imagery', name='map-ready')
required_info = map_pipeline['data']['settings_schema']['required']
print("The required information needed to place an order on this pipeline is: {}".format(required_info))
for info in required_info:
    type_info = map_pipeline['data']['settings_schema']['properties'][info]
    print("The type of data for {} is: {}".format(info, type_info))

#### Create a Tasking Request

Before the tasking request can be submitted, the datetime module will need to be imported to make it easier to get UTC dates in `ISO-8601` format.

In [ ]:
import datetime

now_start = datetime.datetime.utcnow() + datetime.timedelta(minutes=10)
now_time = now_start.isoformat()
end_start = datetime.datetime.utcnow() + datetime.timedelta(days=1)
end_time = end_start.isoformat()
now = now_time.replace(now_time[-7:], "+00:00")
end = end_time.replace(now_time[-7:], "+00:00")
print(now)
print(end)

Having set the beginning and ending times, the tasking template will need to be set up. Using the information gathered from the pipeline request, the template can be constructed. *NOTE: The `inventory_ids` field needs to be included in the tasking request, but no ids need to be passed in. Instead, use the string shown in the list as a placeholder.*

In [ ]:
settings = {"inventory_ids": ["$.id"], "customer_description": "Your description"}
output_config = {"amazon_s3": {"bucket": "yourS3BucketName", "prefix": "your/S3/bucket/prefix/name"}}
notifications = {"type": "email", "target": "your.email@address.com", "level": "INITIAL_FINAL"}

In [ ]:
template = [{
    "pipeline": "imagery/map-ready",
    "template": {
        "settings": settings,
        "output_config": output_config,
        "notifications": [notifications],
        "metadata": {
            "project_id": "Your tasking name"
        }
    }
}]

A geoJSON will also be set in advance to use in the tasking request.

In [ ]:
coordinates = {
    "type": "Polygon", 
    "coordinates": [
        [
            [152.931404, -27.556982],
            [153.172073, -27.556982],
            [153.172073, -27.394936],
            [152.931404, -27.394936],
            [152.931404, -27.556982]
        ]
    ]
}

With our variables set, the tasking request is almost ready to be submitted. First, a validation on the tasking request can be made to make sure that everything is formatted correctly. The recipe value for this example will be `50cm_Color`.

In [ ]:
tasking_validation = mgp_tasking.new_tasking(start_datetime=now, end_datetime=end, aoi_geojson=coordinates, 
                                             recipe='50cm_Color', order_templates=template, validate=True)
print(tasking_validation)

With the validation request passing, the full tasking request can be submitted.

In [ ]:
tasking = mgp_tasking.new_tasking(start_datetime=now, end_datetime=end, aoi_geojson=coordinates, 
                                  recipe='50cm_Color', order_templates=template)
tasking_id = tasking['data']['id']
print(tasking)

#### Viewing Tasking Requests

The tasking request has been submitted and the `tasking_id` has been set to a variable for use further along in the workflow. With the request submitted, the `get_tasking_request` function can be ran to check on the status of the request.

In [ ]:
my_tasking = mgp_tasking.get_tasking_request(tasking_id=tasking_id)
print(my_tasking)

#### Cancel a Tasking Request

In some cases, a user may want to cancel a tasking order before it triggers. This can be acomplished with the `cancel_tasking` funciton.

In [ ]:
cancel_tasking = mgp_tasking.cancel_tasking(tasking_id=tasking_id, reason="Your reason for cancelling")
print(cancel_tasking)